## Multi-Label Classification with BERT
### Taxes Domain
Adapted from the [work](https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613) of Susan Li

If working on Google Colab, it is good to mount your drive and save models there.

In [2]:
#from google.colab import drive
#drive.mount('/content/gdrive')

Mounted at /content/gdrive


Install the transformers package to use BERT and import all necessary libraries.

In [ ]:
#!pip install transformers

In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import numpy as np

from preprocessing import read_data

### Preprocessing data

In [2]:
def preprocess_csv(input_file_name, output_file_name, file_directory="../../data/"):
    """
    cleans a csv file by converting it into a Pandas dataframe and removing unwanted columns, 
    then converts it to another csv file for BERT multi-class classification.
    """
    df = pd.read_csv(file_directory+input_file_name)
    df = df[['Comment', 'Tags']].to_csv(file_directory+output_file_name, index=False)
    return f"{file_directory+output_file_name}"

In [4]:
preprocess_csv('taxes_full.csv', 'taxes_full_df.csv')

'../../data/taxes_full_df.csv'

In [5]:
df = read_data("../../data/taxes_full_df.csv")

In [6]:
df.head()

,Comment,Tags
0,I cannot find a phone number to see what happe...,Owing the CRA
1,How do I cancel installment payments? I always...,Owing the CRA
2,Looking for where to mail payment,Owing the CRA
3,Where to remit my tax payments,Owing the CRA
4,Trying to print out my CERB payments. A type o...,T4 information (T1/T4/T4A/T4E/R1)


In [8]:
## Replace text tags with numbers

labels = df.Tags.unique()

label_dict = {}
for i, label in enumerate(labels):
    label_dict[label] = i
label_dict

{'After filing (notice of assessment/statement of account)': 4,
 'COVID benefits': 9,
 'Contact ': 6,
 'Deadlines/rates': 2,
 'Direct deposit and banking information': 16,
 'Filing for someone else ': 12,
 'Filing from abroad': 8,
 'Income tax form or guide': 3,
 'Less Common Tax Slips (T2202/T3/RC62)': 13,
 'My Account (address change)': 11,
 'Owing the CRA': 0,
 'Page Issues (navigation/links)': 14,
 'Software (Netfile/E-file)': 7,
 'T4 information (T1/T4/T4A/T4E/R1) ': 1,
 'Tax deductions/credits/expenses': 5,
 'Tax refund': 10,
 'pensions and savings (TFSA/RRSP)': 15}

In [9]:
df['label'] = df.Tags.replace(label_dict)

### Splitting the data + Creating the data loader

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

### Initialize BERT Tokenizer and Batch Encoder

In [11]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_train = bert_tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Comment.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_val = bert_tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Comment.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_train['input_ids']
attention_masks_train = encoded_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_val['input_ids']
attention_masks_val = encoded_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


### Create Data Loader

In [12]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_val = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

### Initialize BERT Model to run on GPU

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [14]:
bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
bert_model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [15]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(bert_model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

### Define Evaluation Metrics

In [16]:
from sklearn.metrics import f1_score, accuracy_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy(preds, labels, per_class=True):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    correct_tags = 0
    total_tags = 0
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        correct_tags += (len(y_preds[y_preds==label]))
        total_tags += len(y_true)
        if per_class==True:
          print(f'Class: {label_dict_inverse[label]}')
          print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)} = {round(len(y_preds[y_preds==label])/len(y_true), 4)} \n')

    if per_class==False:
      print(f'Overall Accuracy: {round(correct_tags/total_tags, 4)}')

### Training the model

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    bert_model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = bert_model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [17]:
%%time
    
for epoch in tqdm(range(1, epochs+1)):
    
    bert_model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        bert_model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = bert_model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(bert_model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(bert_model.state_dict(), f'gdrive/MyDrive/COLX_Capstone/model_full_taxes/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    #val_acc = accuracy_score(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 1.5011971297187934
Validation loss: 1.1512743579783564
F1 Score (Weighted): 0.6555477225973735



Epoch 2
Training loss: 0.8645723193887502
Validation loss: 1.0779549186448745
F1 Score (Weighted): 0.7047313221197231



Epoch 3
Training loss: 0.6416517290659068
Validation loss: 1.1529896707996685
F1 Score (Weighted): 0.726181165695539



Epoch 4
Training loss: 0.4743783851357002
Validation loss: 1.166845679303499
F1 Score (Weighted): 0.7515821030729118



Epoch 5
Training loss: 0.3913173821140784
Validation loss: 1.2163273960458818
F1 Score (Weighted): 0.7398814831771868

CPU times: user 10min 20s, sys: 7.55 s, total: 10min 27s
Wall time: 10min 43s
Parser   : 124 ms


### Loading the Model + Checking Accuracy Score

In [18]:
## Load model
bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

bert_model.to(device)

bert_model.load_state_dict(torch.load('gdrive/MyDrive/COLX_Capstone/model_full_taxes/finetuned_BERT_epoch_4.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_val)
accuracy(predictions, true_vals)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Class: Owing the CRA
Accuracy: 11/17 = 0.6471 

Class: T4 information (T1/T4/T4A/T4E/R1) 
Accuracy: 117/135 = 0.8667 

Class: Deadlines/rates
Accuracy: 32/37 = 0.8649 

Class: Income tax form or guide
Accuracy: 106/127 = 0.8346 

Class: After filing (notice of assessment/statement of account)
Accuracy: 10/11 = 0.9091 

Class: Tax deductions/credits/expenses
Accuracy: 23/35 = 0.6571 

Class: Contact 
Accuracy: 27/35 = 0.7714 

Class: Software (Netfile/E-file)
Accuracy: 60/74 = 0.8108 

Class: Filing from abroad
Accuracy: 0/1 = 0.0 

Class: COVID benefits
Accuracy: 9/17 = 0.5294 

Class: Tax refund
Accuracy: 0/2 = 0.0 

Class: My Account (address change)
Accuracy: 14/21 = 0.6667 

Class: Filing for someone else 
Accuracy: 0/2 = 0.0 

Class: Less Common Tax Slips (T2202/T3/RC62)
Accuracy: 1/12 = 0.0833 

Class: Page Issues (navigation/links)
Accuracy: 7/17 = 0.4118 

Class: pensions and savings (TFSA/RRSP)
Accuracy: 0/3 = 0.0 

Class: Direct deposit and banking information
Accuracy: 0/2 =

In [19]:
accuracy(predictions, true_vals, per_class=False)

Overall Accuracy: 0.7609
